In [6]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [7]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

for i in range(len(files)):
    if 'E_no' in files[i]:
        if '__' not in files[i]:
            if 'all' not in files[i]:
                filenames.append(files[i])
                print(i, files[i])

2 Ag_100_u2_RPBE_no001_CO_n1_d9.traj
6 Ag_100_u2_RPBE_no002_CO_n1_d9.traj
14 Ag_110_u2_RPBE_no001_CO_n1_d9.traj
18 Ag_110_u2_RPBE_no002_CO_n1_d9.traj
22 Ag_110_u2_RPBE_no003_CO_n1_d9.traj
26 Ag_110_u2_RPBE_no004_CO_n1_d9.traj
32 Ag_111_u2_RPBE_no001_CO_n1_d9.traj
36 Ag_111_u2_RPBE_no002_CO_n1_d9.traj
40 Ag_111_u2_RPBE_no003_CO_n1_d9.traj


In [8]:
### DELETE ###
for file in filenames:
    try:
        A = make_database(file)
        A.delete_from_database()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Not in database.
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Not in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
Not in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no002_CO_n1_d9.traj
Not in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no003_CO_n1_d9.traj
Not in database.
-----------------------------------------------------------
Ag_110_u2_RPBE_no004_CO_n1_d9.traj
Not in database.
-----------------------------------------------------------
Ag_111_u2_RPBE_no001_CO_n1_d9.traj
Not in database.
-----------------------------------------------------------
Ag_111_u2_RPBE_no002_CO_n1_d9.traj
Not in database.
-----------------------------------------------------------
Ag_111_u2_RPBE_no003_CO_n1_d9.traj
Not in da

In [9]:
### ADD and UPDATE ENERGY ###
for file in filenames:
    try:
        A = make_database(file)
        A.add_to_database()
        A.update_Energy()
        A.update_adsorbates_correlation()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Not in database.
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
One post: 5e68567769846abba1a74212

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Not in database.
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
One post: 5e68567769846abba1a74214

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
Not in database.
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
One post: 5e68567869846abba1a74216

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlatioin is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Ag_110_u2_

In [37]:
for file in filenames:
    try:
        A = make_database(file)
        data = A.check_database()
        print(data['name'], data['minimum_distance'], data['ads_dist2'], data['ads_dist3'])
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Ag_100_u2_RPBE_no001_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Ag_100_u2_RPBE_no002_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Ag_100_u2_RPBE_no003_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no001_CO_n1_d9.traj
Ag_110_u2_RPBE_no001_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no002_CO_n1_d9.traj
Ag_110_u2_RPBE_no002_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no003_CO_n1_d9.traj
Ag_110_u2_RPBE_no003_CO_n1_d9.traj None None None
-----------------------------------------------------------
Ag_110_u2_RPBE_no004_CO_n1_d9.traj
Ag_110_u2_RPBE_no004_CO_n1_d9.traj 

-----------------------------------------------------------
Cu_111_u2_RPBE_no002_CO_n1_d8.traj
Cu_111_u2_RPBE_no002_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_111_u2_RPBE_no003_CO_n1_d8.traj
Cu_111_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_111_u2_RPBE_no004_CO_n1_d8.traj
Cu_111_u2_RPBE_no004_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no001_CO_n1_d8.traj
Cu_211_u2_RPBE_no001_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no002_CO_n1_d8.traj
Cu_211_u2_RPBE_no002_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no003_CO_n1_d8.traj
Cu_211_u2_RPBE_no003_CO_n1_d8.traj None None None
-----------------------------------------------------------
Cu_211_u2_RPBE_no004_CO_n1_d8.traj
Cu_211_u2_RPBE_no004_CO_n1_d8.traj 

In [21]:
# for file in filenames:
#     try:
#         A = make_database(file)
#         print(A.check_database()['minimum_distance'])
#     except:
#         print(file, 'Some error occured.')

In [22]:
file = 'Cu_100_u2_RPBE_no006_CO_n4_d3.traj'
A = make_database(file)
A.check_database()

No file named Cu_100_u2_RPBE_no006_CO_n4_d3.traj in init
No file named Cu_100_u2_RPBE_no006_CO_n4_d3.traj in database
-----------------------------------------------------------
Cu_100_u2_RPBE_no006_CO_n4_d3.traj
Not in database.


In [47]:
query('Cu_110_u2_RPBE_no003_CO_n3_d3.traj','local')

Atoms(symbols='Cu16COCOCO', pbc=[True, True, False], cell=[7.354476072672618, 5.200399903061017, 23.900299927295762], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))